# Suchalgorithmen am Beispiel 8-Puzzle

![quadrate](img/such_bild1.png)

#### Breitensuche (bfs)

```
Initialisiere frontier als Queue mit dem startstate
Initialisiere prev als dictionary der Vorgänger
Der Vorgänger von startstate ist None
nrExplored = 0

solange frontier nicht leer:
    hole state aus frontier
    nrExplored += 1
    
    wenn goalTest(state):
        return dictionary mit Vorgängern, nrExplored
    
    für jedes v aus nextstates(state):
        wenn v kein key in prev:
            füge v in die frontier ein.
            merke state als Vorgänger von v
```

In [5]:
startstate = (7,2,4,5,0,6,8,3,1)
goalstate  = (0,1,2,3,4,5,6,7,8) 

from collections import deque
def bfs(s): 
    frontier = deque([s])
    prev = {s:None}
    nrExplored = 0
    
    while frontier:
        state = frontier.popleft()
        nrExplored += 1
        if goaltest(state):
            return prev, nrExplored
        for v in nextstates(state):
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, nrExplored

def dfs(s): 
    frontier = [s]
    prev = {s:None}
    nrExplored = 0
    
    while frontier:
        state = frontier.pop()
        nrExplored += 1
        if goaltest(state):
            return prev, nrExplored 
        nxt = nextstates(state)
        nxt.reverse()
        for v in nxt:
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None, nrExplored


def goaltest(state):
    return state == goalstate

def nextstates(state):
    if state[0] == 0:   return [swap(state,0,3),swap(state,0,1)]
    elif state[1] == 0: return [swap(state,1,4),swap(state,1,0),swap(state,1,2)]
    elif state[2] == 0: return [swap(state,2,5),swap(state,2,1)]
    elif state[3] == 0: return [swap(state,3,0),swap(state,3,6),swap(state,3,4)]
    elif state[4] == 0: return [swap(state,4,1),swap(state,4,7),swap(state,4,3),swap(state,4,5)]
    elif state[5] == 0: return [swap(state,5,2),swap(state,5,8),swap(state,5,4)]
    elif state[6] == 0: return [swap(state,6,3),swap(state,6,7)]
    elif state[7] == 0: return [swap(state,7,4),swap(state,7,6),swap(state,7,8)]
    elif state[8] == 0: return [swap(state,8,5),swap(state,8,7)]

def swap(state, i, j):
    tmp = list(state)
    tmp[i],tmp[j] = tmp[j],tmp[i]
    return tuple(tmp)

def reconstructPath(prev):
    s = (0,1,2,3,4,5,6,7,8)
    tmp = []
    while prev[s] is not None:
        i = s.index(0)
        ip = prev[s].index(0)
        if i == ip - 1: tmp.append('left')
        elif i == ip + 1: tmp.append('right')
        elif i == ip - 3: tmp.append('up')
        elif i == ip + 3: tmp.append('down')
        s = prev[s]
    tmp.reverse()
    return ' '.join(tmp), len(tmp)

In [6]:
%%time
startstate = (7,2,4,5,0,6,8,3,1)
prev, nrExplored = bfs(startstate)
print(nrExplored)
reconstructPath(prev)
 


171712
Wall time: 570 ms


('left up right down down left up right right up left left down right right down left up right up left down down left up up',
 26)

In [8]:
%%time

startstate = (7,2,4,5,0,6,8,3,1)
prev, nrExplored  = dfs(startstate)
print(nrExplored)
_, laenge = reconstructPath(prev)
print(laenge)
 

114208
62140
Wall time: 393 ms


#### Heuristik

In [74]:
state      = (3,1,4,5,0,6,8,7,2)
def h(s):
    return sum([a!=b for a,b in zip((0,1,2,3,4,5,6,7,8),s) ])
h(state)

7

#### Greedy

In [75]:
from heapq import heappop, heappush
def greedy(s): 
    frontier = [(h(s),s)]
    prev = {s:None}
    nrExplored = 0
    
    while frontier:
        _, state = heappop(frontier) 
        nrExplored += 1
        if goaltest(state):
            return prev, nrExplored
        for v in nextstates(state):
            if v not in prev:
                heappush(frontier,(h(v),v))
                prev[v] = state
    return None, nrExplored


In [76]:
 %%time
startstate = (7,2,4,5,0,6,8,3,1)
prev, nrExplored = greedy(startstate)
print(nrExplored)
reconstructPath(prev)

489
Wall time: 6.98 ms


('up left down right up left down right up right down left down left up right up right down down left up up right down left down left up up right right down left left up right down right up left down left down right up up left down right down left up up right down right up left left down right right up left down left up right down left up right right down left left up right down right up left down left down right up up left down right down left up up right down left down right up up left down right down left up up',
 110)

#### A-Star

In [77]:
from heapq import heappop, heappush
def astar(s): 
    g = {s:0}
    frontier = [(h(s)+g[s],s)]
    prev = {s:None}
    nrExplored = 0
   
    
    while frontier:
        _, state = heappop(frontier) 
        nrExplored += 1
        if goaltest(state):
            return prev, nrExplored
        for v in nextstates(state):
            gg = g[state] + 1
            if v not in prev or gg < g[v]:
                g[v] = gg
                heappush(frontier,(h(v)+g[v],v))
                prev[v] = state
    return None, nrExplored

In [82]:
%%time
startstate = (7,2,4,5,0,6,8,3,1)
prev, nrExplored = astar(startstate)
print(nrExplored)
reconstructPath(prev)

33167
Wall time: 345 ms


('left up right down right down left left up right right down left left up right right up left left down right right up left left',
 26)

In [83]:
def h(state):
    mh = 0
    for k in range(9):
        if state[k] != 0:
            z1,s1 = k//3,k%3
            z2,s2 = state[k]//3,state[k]%3
            mh += abs(z1-z2)+abs(s1-s2)
    return mh

In [84]:
%%time
startstate = (7,2,4,5,0,6,8,3,1)
prev, nrExplored = astar(startstate)
print(nrExplored)
reconstructPath(prev)

2034
Wall time: 45.9 ms


('left up right down right down left left up right right down left left up right right up left left down right right up left left',
 26)